# Text analysis with Azure Open AI

In [1]:
!pip install openai==0.28

In [11]:
import datetime
import openai
import os
import sys

from dotenv import load_dotenv

In [12]:
load_dotenv("azure.env")

# Azure Open AI
openai.api_type: str = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

print("Open AI version:", openai.__version__)

Open AI version: 0.28.0


## Function

In [13]:
model = "_testing_text_davinci"

In [14]:
def azure_openai(prompt, temperature=0.7):
    """
    Get Azure Open AI results
    """
    prompt = prompt + "\n" + text

    results = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=800,
    )

    answer = results["choices"][0]["text"].strip("\n")

    return answer

## PII analysis

In [15]:
text = """
Parker Doe has repaid all of their loans as of 2020-04-25.
Their SSN is 859-98-0987. To contact them, use their phone number
555-555-5555. They are originally from Brazil and have Brazilian CPF number 998.214.865-68
"""

print(text)


Parker Doe has repaid all of their loans as of 2020-04-25.
Their SSN is 859-98-0987. To contact them, use their phone number
555-555-5555. They are originally from Brazil and have Brazilian CPF number 998.214.865-68



In [18]:
answer = azure_openai("What are the Personally identifiable information in this text?")
print(answer)

Please reach out to them to know more about their life interests.

*You can also see their previous area code information here.

Parker Doe was born in 1984-10-25. Their current address is 1 PND City, Brazil. Their previous address was in Boston, MA, 02116. They lived there from 2012-09-04 to 2020-05-01.

*You can also see their previous addresses here.

Parker Doe is an individual connected to the following individuals: Ann Doe, David Doe, Lucy Doe, Alice Doe, Andrew Doe, Michael Doe, James Doe, Anna Doe, John Doe, Elizabeth Doe, Peter Doe, Emma Doe, Sophie Doe, Grace Doe, Carol Doe, Helen Doe, Mary Doe, Lauren Doe, Amy Doe, Ben Doe, William Doe, Jonathan Doe, Robert Doe, Richard Doe, Mark Doe, Gregory Doe, Chris Doe, Jim Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe, John Doe

Parker Doe is an individual with vegan dietary preferences.

Parker Doe is an individual with a Republ

In [17]:
answer = azure_openai(
    "What are the Personally identifiable information in this text? Save in a json file"
)
json = answer
print(json)

Their email address is panuska@acme.com. They are not currently on the Do Not Call list.

You have 6 days to report their phone number fraud. After that, they can do whatever they want with your bank account.

You can report this text as fraud to the FTC here.

Is this Parker Doe? Claim this profile!


# Resume analysis with Azure Open AI

In [19]:
model = "_testing_gpt35turbo"

## Reading the resume

In [27]:
resume_file = "resume.txt"

In [21]:
with open(resume_file, "r") as f:
    doc = f.readlines()

doc = " ".join(doc)

In [22]:
print(doc[:500])

Bikash Agrawal
 Phone: (47) 456 6666
 Email: bikash.agrawal@simplifai.com
 Website: www.simplifai.ai
 Linkedin: https://www.linkedin.com/in/bikash_agrawal
 Address: 1054 Oslo
 Date of birth: 10-Mar-1984
 Place of Birth: Nepal
 Nationality: Nepal
 
 
 Objective
 Data Scientist with PhD in Big Data and 15+ years of industrial experience. Seven years of working experience in Data Analysis team of LIGO Scientific Collaboration. Over ten years of successful research experience in both theoretical and


In [23]:
def resume_qna(document, prompt, model):
    """
    Analysing resume with Azure Open AI
    """
    content = f""" {document}
      \n###
      \n{prompt}
    """

    messages = [{"role": "user", "content": content}]
    
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0,
        max_tokens=800,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop="###",
    )

    return response

In [24]:
prompt = "Extract the key sections from the resume above into json."

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 {
  "name": "Bikash Agrawal",
  "phone": "(47) 456 6666",
  "email": "bikash.agrawal@simplifai.com",
  "website": "www.simplifai.ai",
  "linkedin": "https://www.linkedin.com/in/bikash_agrawal",
  "address": "1054 Oslo",
  "date_of_birth": "10-Mar-1984",
  "place_of_birth": "Nepal",
  "nationality": "Nepal",
  "objective": "Data Scientist with PhD in Big Data and 15+ years of industrial experience. Seven years of working experience in Data Analysis team of LIGO Scientific Collaboration. Over ten years of successful research experience in both theoretical and datascience. Strong problem-solving and analytical skills. Advanced programming proficiency. Certified in Data Analysis and Machine Learning on Azure and AWS services.",
  "skills": [
    "Azure",
    "Azure AI",
    "Azure ML",
    "Data Mining",
    "Data Analysis",
    "Machine Learning",
    "Python",
    "R",
    "Sphinx",
    "Mathematica",
    "Maple",
    "GIT",
    "Azure Custom Vision",
    "Azure Open AI",
    "

In [25]:

prompt = "Can you describe this resume in 3 lines?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 This resume belongs to Bikash Agrawal, a Data Scientist with a PhD in Big Data and over 15 years of industrial experience. They have a strong background in data analysis and machine learning, with expertise in Azure, Python, and R. They have worked as a Data Scientist, Data Science Manager, and Datascience Trainer in various organizations.


In [26]:
prompt = "What are the 10 keywords of this resume?"

answer = resume_qna(doc, prompt, model)
print("\033[1;31;34mAnswer:\n\n", answer["choices"][0]["message"]["content"])

Answer:

 1. Data Scientist
2. PhD
3. Big Data
4. Data Analysis
5. Machine Learning
6. Python
7. Azure
8. Datascience
9. Manager
10. Programming
